# Assignment 3: Unsupervised Learning

In this assignment you will:
* Implement K-means clustering and use it for color-quantization of images
* Train a MoG model by implementing the EM algorithm. Compare to K-means on the same test data. 
* Use PCA for image compression

# Part 1: K-Means Clustering

## Setup

First import the required packages and do some setup:

In [ ]:
from __future__ import print_function

import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import misc
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from assignment3 import kmeans, kmeans_colors, em_mog, em_segmentation, getEigenImages

# Set default parameters for plots
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'jet'

%load_ext autoreload
%autoreload 2



Let's generate some test-data for you to test your implementation of the k-means and EM algorithm. Note that we generate gaussian blobs with non-isotropic covariance matrices. 

In [ ]:
# Generate samples of 4 different Gaussians
rv1 = multivariate_normal.rvs([4.0, -3.], [[4.0, 1.5], [1.5, 1.1]], size=400)
rv2 = multivariate_normal.rvs([4.0, 3.5], [[1.0, 2.3], [0.8, 2.75]], size=500)
rv3 = multivariate_normal.rvs([-3.5, -4.0], [[0.5, 1.0], [0.3, 3.]], size=600)
rv4 = multivariate_normal.rvs([-3., 3.], [[2.5, 0.1], [2.3, 1.5]], size=700)

# Concatenate the samples and create corresponding labels
X = np.concatenate([rv1, rv2, rv3, rv4], 0)
y = np.array([0]*400+[1]*500+[2]*600+[3]*700)

# Plot the test data
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title("Test Blobs")
plt.show()



## Exercise 1: K-Means Clustering [15 Points]

You will now implement the standard k-means clustering algorithm and test it on the generated test data. 

**TODO**: Implement the k-means clustering algorithm in ***kmeans.py*** according to specs and test your implementation with the provided test data.


In [ ]:
# Test your implementation
centers, assign = kmeans(X, 4)
plt.clf()
plt.scatter(X[:, 0], X[:, 1], c=assign)
plt.scatter(centers[:, 0], centers[:, 1], c='r', marker='x')
plt.title("Predicted clustering")
plt.show()


## Exercise 2: Questions about K-means [10 Points]

* Does k-means always result in the same clustering? Why?

***Your Answer:***

* Would you expect K-means to work better in the case of data generated from isotropic Gaussian distributions? Why?

***Your Answer:***

## Exercise 3: Using k-means for image compression [10 Points]

You will now use K-means to cluster pixel values of an image and use the cluster assignments as a way to quantize/compress the color-space of the image. Replace each pixel with the mean of the assigned cluster.
You should use the implementation provided through sklearn in this exercise ([sklearn.cluster.KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)).

Feel free to test the algorithm on your own images and with different numbers of clusters.

**TODO**: Implement the color-quantization through k-means in ***kmeans_colors.py*** according to specs. 


In [ ]:
# Load and show test image
img = misc.imread('lena.png')
plt.imshow(img)
plt.title("Original image")
plt.show()

# Compute quantized image
k = 4
img_cl = kmeans_colors(img, k)

# Show the quantized image
plt.imshow(img_cl)
plt.title("Quantized image")
plt.show()

# Part 2: Expectation Maximization

In the second part of the assignment you will implement the EM algorithm for Mixture of Gaussian (MoG) models. You will then use this model on the problem of image-segmentation.


## Exercise 4: EM for MoG [25 Points]

**TODO:** Implement the **EM** algorithm for Mixture of Gaussian models in **em_mog.py** according to specs. Test your implementation on the provided test data.  

In [ ]:
# Test your implementation
phi, mu, sigma, w = em_mog(X, 4)
plt.clf()
plt.scatter(X[:, 0], X[:, 1], c=np.argmax(w, 1))
plt.scatter(mu[:, 0], mu[:, 1], c='r', marker='x')
plt.title("Predicted clustering")
plt.show()


## Exercise 5: Questions about MoG [10 Points]

* Does the MoG model perform better than K-means on the provided test data? Why?

***Your Answer:***

* What are advantages and disadvantages of MoG vs. K-means?

***Your Answer:***

## Exercise 6: EigenFaces with PCA [5 Points]
In this exercise you will implement an image compression method using PCA. 
Below are the steps that you need to do:
* Stack all the images of faces in the design matrix 
* Perform SVD (Singular Value Decomposition) of the design matrix
* Get the _right_ eigenvectors corresponding to the N largest eigenvalues
* Project a test image onto the basis of eigenvectors from the previous step

In order to reconstruct the compressed image you need to compute the linear combination of the projection coefficients and corresponding _right_ eigenvectors. For more details please visit: https://en.wikipedia.org/wiki/Principal_component_analysis

Loading the dataset of faces

In [ ]:
data = loadmat('faces.mat')
labels = np.squeeze(data['Labels'])
data = data['Data']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1)
num_train = X_train.shape[0]
num_test = X_test.shape[0]

# Zero centering data
scaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=False)
scaler.fit(data[labels==1])

# Selecting only faces
X_faces_train = scaler.transform(X_train[y_train==1])
X_faces_test = scaler.transform(X_test[y_test==1])



Displaying some of the samples

In [ ]:
for i in range(1,10):
    plt.subplot(3, 3, i)
    plt.imshow(np.reshape(X_faces_train[i-1,:], [24, 24], order='F'), cmap='gray')
    plt.axis('off')

plt.show()
plt.close()


Apply PCA to the training images

In [ ]:

U=None 
S=None 
Vh=None

#######################################################################
# TODO:                                                               #
#      Perform the singular value decomposition (SVD) of the face     #
#      data matrix and    #
#      extract the eigenvectors                                       #
#      Input: X_faces_train
#      Output: U, S, and Vh matrices of the SVD
#######################################################################

pass

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

Display some of the EigenFaces

In [ ]:
eigenface = np.reshape(Vh, [-1, 24, 24], order='F')

N=9

for i in range(1,N+1):
    plt.subplot(int(np.sqrt(N)), int(np.sqrt(N)), i)
    plt.imshow(eigenface[i-1], cmap='gray')
    plt.axis('off')

plt.show()
plt.close()

## Exercise 7: Compression using PCA [10 Points]

Compress the test set images using eigenvectors computed in the previous exercise

**TODO:** Implement the **Compression with PCA** in **getEigenImages.py** according to specs.

In [ ]:
# Selecting some number of EigenFaces
num_components = 200
basis = Vh[:num_components,:].T

eigen_coefficients, reconstruction = getEigenImages(X_faces_test, basis)

Test your implementation on the provided test data. Display the reconstructed and original images side-by-side

In [ ]:
reconstructed_images = np.reshape(reconstruction,[-1, 24,24], order='F')
test_images = np.reshape(X_faces_test,[-1, 24,24], order='F')

plt.rcParams['figure.figsize'] = (16.0, 3.0)

num_faces = 9

for i in range(1,num_faces+1):
    plt.subplot(2, num_faces, i)
    plt.imshow(test_images[i-1,:,:], cmap='gray')
    plt.axis('off')
    plt.subplot(2, num_faces, num_faces + i)
    plt.imshow(reconstructed_images[i-1,:,:], cmap='gray')
    plt.axis('off')

plt.show()
plt.close()

## Exercise 8: Evaluating the Quality of the Compression [5 Points]

Compress the images from the test set using different numbers of eigenvectors. Measure the quality of the reconstructions using MSE. Plot the dependency of MSE on the number of eigenvectors used for the reconstruction. Recall that the MSE between two images is defined as

$$MSE(X,Y)= \frac { 1 } { m n } \sum _ { i = 0 } ^ { m - 1 } \sum _ { j = 0 } ^ { n - 1 } [ X ( i , j ) - Y ( i , j ) ] ^ { 2 }$$

In [ ]:
num_eigen_vectors_list = range(24,577,24)
mse_list = []

for num_eigen_vectors in num_eigen_vectors_list:
    basis = Vh[:num_eigen_vectors,:].T
    
    eigen_coefficients, reconstruction = getEigenImages(X_faces_test, basis)
    
    #######################################################################
    # TODO:                                                               #
    #      Compute MSE between reconstruction and original image          #                                  
    #######################################################################
    
    pass
    
    #######################################################################
    #                         END OF YOUR CODE                            #
    #######################################################################
    
fig, ax = plt.subplots()
ax.plot(num_eigen_vectors_list, mse_list)

ax.set(xlabel='Number of EigenFaces', ylabel='MSE',
       title='Dependency of MSE on the Number of EigenFaces')
ax.grid()
plt.show()

## Exercise 9: EigenFace with PCA [10 Points]

* In the lecture notes you were computing the eigenvectors of the data covariance matrix, while in assignment you computed them by directly performing SVD on the design matrix. What is the difference between the two approaches? Which approach is better? Justify your answer.

***Your Answer:***


* Discuss the quality of the reconstructions and advantages of using PCA for compression.

***Your Answer:***